Frozen Lake
===
The goal of this computer assignment is to get familiar with OpenAI Gym, implement value iteration and policy iteration.

Problem Description
---
OpenAI Gym is a toolkit for developing and comparing reinforcement learning algorithms. It supports teaching agents everything from walking to playing games like Pong or Pinball. For more information visit https://gym.openai.com.

In this computer assigment, you'll get familiar with Frozen Lake environment and implement value and policy iteration algorithms. Frozen Lake is an environment where the agent controls the movement of a character in a grid world. Some tiles of the grid are walkable, and others lead to the agent falling into the water. Additionally, the movement direction of the agent is uncertain and only partially depends on the chosen direction. The agent is rewarded for finding a walkable path to a goal tile. For more information please visit https://gym.openai.com/envs/FrozenLake8x8-v0/.

Your Job
---
1. Get started with gym by following the steps here https://gym.openai.com/docs/.
2. Read https://gym.openai.com/envs/FrozenLake8x8-v0/ to get familiar with the environment, states, reward function, etc.
3. Implement the $\texttt{value_iteration}$ function below.
4. Implement the $\texttt{policy_iteration}$ function below.
5. Answer the questions (By double click on the cell you can edit the cell and put your answer below each question).

In [12]:
from __future__ import print_function
from __future__ import division

import numpy as np
import gym
from gym import wrappers
import time

def run_episode(env, policy, gamma, render = False):
    """ Evaluates policy by using it to run an episode and finding its
    total reward.
    args:
    env: gym environment.
    policy: the policy to be used.
    gamma: discount factor.
    render: boolean to turn rendering on/off.
    returns:
    total reward: real value of the total reward recieved by agent under policy.
    """
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        obs, reward, done , _ = env.step(int(policy[obs]))
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward


def evaluate_policy(env, policy, gamma,  n = 100):
    """ Evaluates a policy by running it n times.
    returns:
    average total reward
    """
    scores = [
            run_episode(env, policy, gamma=gamma, render = False)
            for _ in range(n)]
    return np.mean(scores)

def extract_policy(v, gamma):
    """ Extract the policy given a value-function """
    policy = np.zeros(env.nS)
    for s in range(env.nS):
        q_sa = np.zeros(env.action_space.n)
        for a in range(env.action_space.n):
            for next_sr in env.P[s][a]:
                # next_sr is a tuple of (probability, next state, reward, done)
                p, s_, r, _ = next_sr
                q_sa[a] += (p * (r + gamma * v[s_]))
        policy[s] = np.argmax(q_sa)
    return policy


def value_iteration(env, gamma, epsilon=1e-20, max_iterations=100000):
    """
    This function implements value iteration algorithm for the infinite
    horizon discounted MDPs. If the sup norm of v_k - v_{k-1} is less than
    epsilon or number of iterations reaches max_iterations, it should return
    the value function.
    """
    start = time.time()
    v = np.zeros(env.nS)  # initialize value-function
    ########################### Your Code Here ###########################
    # Hint: see implementation of extract_policy
    for i in range(max_iterations):
        v1 = np.copy(v)
        for s in range(env.nS):
            Q = np.zeros(env.action_space.n)
            for a in range(env.action_space.n):
                for next_sr in env.P[s][a]:
                    p, s_, r, _ = next_sr
                    Q[a] += (p * (r + gamma * v1[s_]))
            v[s] = np.max(Q)
        if(np.sum(np.fabs(v1-v)) <= epsilon):
            print("Iteration step = ",i)
            break
    ########################### End of your code #########################
    end = time.time()
    print("Value iteration took {0} seconds.".format(end - start))
    return v

if __name__ == '__main__':
    np.random.seed(1111)
    env_name  = 'FrozenLake8x8-v0'
    for gamma in [.9, .95, .99, .9999, 1]:
        print("-"*10, "Gamma={0}".format(gamma) ,"-"*10)
        env = gym.make(env_name)
        optimal_v = value_iteration(env, gamma);
        policy = extract_policy(optimal_v, gamma)
        policy_score = evaluate_policy(env, policy, gamma, n=1000)
        print('Average score = ', policy_score)

---------- Gamma=0.9 ----------
Iteration step =  314
Value iteration took 0.417464017868042 seconds.
Average score =  0.00582952534124281
---------- Gamma=0.95 ----------
Iteration step =  496
Value iteration took 0.5441701412200928 seconds.
Average score =  0.04908442798056828
---------- Gamma=0.99 ----------
Iteration step =  1125
Value iteration took 1.4153997898101807 seconds.
Average score =  0.41152692263539387
---------- Gamma=0.9999 ----------
Iteration step =  2335
Value iteration took 2.480024814605713 seconds.
Average score =  0.8707724315584127
---------- Gamma=1 ----------
Iteration step =  2356
Value iteration took 2.50634503364563 seconds.
Average score =  0.876


Policy Iteration
---

In [13]:
def compute_policy_v(env, policy, gamma):
    """ Iteratively evaluate the value-function under policy.
    Alternatively, we could formulate a set of linear equations in iterms of v[s] 
    and solve them to find the value function.
    """
    v = np.zeros(env.nS)
    eps = 1e-10
    while True:
        prev_v = np.copy(v)
        for s in range(env.nS):
            policy_a = policy[s]
            v[s] = sum([p * (r + gamma * prev_v[s_]) for p, s_, r, _ in env.P[s][policy_a]])
        if (np.sum((np.fabs(prev_v - v))) <= eps):
            # value converged
            break
    return v

def policy_iteration(env, gamma, max_iterations=100000):
    """
    This function implements policy iteration algorithm.
    """
    start = time.time()
    policy = np.random.choice(env.nA, size=(env.nS))  # initialize a random policy
    ########################### Your Code Here ###########################
    for i in range(max_iterations):
        v1 = compute_policy_v(env, policy, gamma)
        new_policy = extract_policy(v1,gamma)
        if(np.all(policy == new_policy)):
            print("iteration step = ",i)
            break
        policy = new_policy
    ########################### End of your code #########################
    end = time.time()
    print("Policy iteration took {0} seconds.".format(end - start))
    return policy


if __name__ == '__main__':
    np.random.seed(1111)
    env_name  = 'FrozenLake8x8-v0'
    for gamma in [.9, .95, .99, .9999, 1]:
        print("-"*10, "Gamma={0}".format(gamma) ,"-"*10)
        env = gym.make(env_name)
        optimal_policy = policy_iteration(env, gamma=gamma)
        scores = evaluate_policy(env, optimal_policy, gamma=gamma)
        print('Average scores = ', np.mean(scores))

---------- Gamma=0.9 ----------
iteration step =  4
Policy iteration took 0.17940711975097656 seconds.
Average scores =  0.00839832646151676
---------- Gamma=0.95 ----------
iteration step =  2
Policy iteration took 0.17818713188171387 seconds.
Average scores =  0.041445836926981
---------- Gamma=0.99 ----------
iteration step =  7
Policy iteration took 0.97104811668396 seconds.
Average scores =  0.42107150393173565
---------- Gamma=0.9999 ----------
iteration step =  11
Policy iteration took 3.004136085510254 seconds.
Average scores =  0.911345348434091
---------- Gamma=1 ----------
iteration step =  5
Policy iteration took 1.7235159873962402 seconds.
Average scores =  0.84


Questions
--- 

#### 1. How many iterations did it take for the value iteration to converge? How about policy iteration?
Value iteration:                         Policy iteration:
Gamma        Iteration step              Gamma        Iteration step
0.9          314                         0.9          4
0.95         496                         0.95         2
0.99         1125                        0.99         7
0.9999       2335                        0.9999       11
1            2356                        1            5


#### 2. How much time did it take for the value iteration to converge? How about the policy iteration?
Value iteration:            
Gamma        Time(seconds)  
0.9          0.417464017868042 seconds.            
0.95         0.5441701412200928 seconds.            
0.99         1.4153997898101807 seconds.           
0.9999       2.480024814605713 seconds.           
1            2.50634503364563 seconds.           

Policy iteration:            
Gamma        Time(seconds)  
0.9          0.17940711975097656 seconds.            
0.95         0.17818713188171387 seconds.            
0.99         0.97104811668396 seconds.           
0.9999       3.004136085510254 seconds.           
1            1.7235159873962402 seconds.  


#### 3. Which algorithm is faster? Why?
Policy algorithm is faster. Since in policy iteration, the inf operation is just finding the inferior of $c(x,u)$. The other part $\gamma\sum_{x^{'}}P_{xx^{'}}(\pi_{n}(x))V^{\pi_n}(x^{'})$ is fixed in each step. However, in value iteration, we need to find the inferior of $c(x,u)+\gamma\sum_{x^{'}}P_{xx^{'}}(u)V_{k}(x^{'})$. It takes more time to calculate the latter function.

#### 4. How does the average score change as $\gamma$ gets closer to 1? Why?
The average score will be increasing as the $\gamma$ gets closer to 1, since the average score is calculated by the average of total_reward $\sum\gamma^kc^*(x_ku_k)$, which is a monotonic function of $\gamma$.


